## Attention 신경망 구현 및 학습

In [1]:
import random
import tensorflow as tf
from konlpy.tag import Okt

## 하이퍼 파라미터

In [2]:
EPOCHS = 200
NUM_WORDS = 2000

## Encoder

In [3]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        H, h, c = self.lstm(x)
        return H, h, c

## Decoder

In [4]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.att = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        x, s0, c0, H = inputs
        x = self.emb(x)
        S, h, c = self.lstm(x, initial_state=[s0, c0])
        
        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)
        A = self.att([S_, H])
        y = tf.concat([S, A], axis=-1)
        
        return self.dense(y), h, c

## Seq2seq

In [5]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos

    def call(self, inputs, training=False, mask=None):
        if training is True:
            x, y = inputs
            H, h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c, H))
            return y
        else:
            x = inputs
            H, h, c = self.enc(x)
            
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, 64)

            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c, H])
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

In [6]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 데이터셋 준비


In [7]:
dataset_file = 'chatbot_data.csv' # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

/home/jeyong/.conda/envs/tf2/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [8]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [9]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

W0922 10:39:47.300251 139735013209920 deprecation.py:323] From /home/jeyong/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1220: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1, Loss: 2.967951536178589, Accuracy: 83.13752746582031
Epoch 2, Loss: 0.6947985887527466, Accuracy: 90.3900375366211
Epoch 3, Loss: 0.5832685828208923, Accuracy: 90.45269775390625
Epoch 4, Loss: 0.557149350643158, Accuracy: 91.1614990234375
Epoch 5, Loss: 0.5447879433631897, Accuracy: 91.14974975585938
Epoch 6, Loss: 0.5346992015838623, Accuracy: 91.13800048828125
Epoch 7, Loss: 0.5256290435791016, Accuracy: 91.15757751464844
Epoch 8, Loss: 0.5153306126594543, Accuracy: 91.19282531738281
Epoch 9, Loss: 0.5023238062858582, Accuracy: 91.26722717285156
Epoch 10, Loss: 0.48601916432380676, Accuracy: 91.33771514892578
Epoch 11, Loss: 0.4716322720050812, Accuracy: 91.56877136230469
Epoch 12, Loss: 0.46000903844833374, Accuracy: 91.91337585449219
Epoch 13, Loss: 0.45097821950912476, Accuracy: 92.10917663574219
Epoch 14, Loss: 0.4431022107601166, Accuracy: 92.28148651123047
Epoch 15, Loss: 0.4359654188156128, Accuracy: 92.40287780761719
Epoch 16, Loss: 0.4292660057544708, Accuracy: 92.4

Epoch 128, Loss: 0.016210924834012985, Accuracy: 99.81202697753906
Epoch 129, Loss: 0.01546811405569315, Accuracy: 99.80811309814453
Epoch 130, Loss: 0.014785263687372208, Accuracy: 99.81202697753906
Epoch 131, Loss: 0.014191846363246441, Accuracy: 99.81594848632812
Epoch 132, Loss: 0.013697736896574497, Accuracy: 99.83552551269531
Epoch 133, Loss: 0.013106673955917358, Accuracy: 99.83161163330078
Epoch 134, Loss: 0.012764694169163704, Accuracy: 99.83943939208984
Epoch 135, Loss: 0.012378379702568054, Accuracy: 99.86685180664062
Epoch 136, Loss: 0.0119987977668643, Accuracy: 99.8629379272461
Epoch 137, Loss: 0.011909674853086472, Accuracy: 99.87077331542969
Epoch 138, Loss: 0.011692952364683151, Accuracy: 99.8629379272461
Epoch 139, Loss: 0.013077173382043839, Accuracy: 99.85511016845703
Epoch 140, Loss: 0.01248419750481844, Accuracy: 99.83161163330078
Epoch 141, Loss: 0.012831509113311768, Accuracy: 99.83552551269531
Epoch 142, Loss: 0.013471788726747036, Accuracy: 99.86685180664062
E

## 테스트 루프

In [10]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q: ', test_text)
    print('a: ', gt_text)
    print('p: ', texts)

_
q:  ['여기 기프티콘 되죠 \n']
a:  ['\t 네 현금영수증 해드릴까 요 \n']
p:  ['아메리카노 기프티콘 으로 가격 머그컵 머그컵 에 드리도록 되어있습니다 \n']
_
q:  ['네 에 테이크 아웃 도 가능한가요 \n']
a:  ['\t 네 로 오시 면 테이크 아웃 잔 에 담아 드려요 \n']
p:  ['아뇨 매장 말씀 머그컵 한 들 가 제일 잘나가요 \n']
_
q:  ['아메리카노 톨 사이즈 로 주세요 \n']
a:  ['\t 따뜻한 거 로 드릴 까요 \n']
p:  ['네 사이즈 는 어떤 걸 로 드릴 까요 \n']
_
q:  ['진동 을 따로 주시나요 \n']
a:  ['\t 주 번호 로 드리겠습니다 \n']
p:  ['네 담아 드립니다 \n']
_
q:  ['자리 있나요 \n']
a:  ['\t 네 있습니다 \n']
p:  ['아니요 치즈케이크 는 지금 없어요 \n']
_
q:  ['그럼 루이보스 밀크 티 하나 \n']
a:  ['\t 네 알겠습니다 \n']
p:  ['네 아메리카노 아메리카노 아메리카노 아메리카노 로 드리겠습니다 \n']
_
q:  ['다음 에 무료 로 하고 엔 도장 찍어주세요 \n']
a:  ['\t 네 \n']
p:  ['네 잠시 만 기다려주세요 \n']
_
q:  ['아메리카노 한 잔 에 얼마 죠 \n']
a:  ['\t 입니다 \n']
p:  ['4000원 입니다 \n']
_
q:  ['얼마나 \n']
a:  ['\t 바로 만들어 드릴게요 \n']
p:  ['10분 정도 걸려요 \n']
_
q:  ['카푸치노 는 로 주시 고 아메리카노 는 로 \n']
a:  ['\t 네 더 없으세요 \n']
p:  ['네 치즈케이크 는 치즈케이크 로 드릴 까요 \n']
_
q:  ['아메리카노 는 어떤 종류 가 있나요 \n']
a:  ['\t 디카 페인 과 기본 아메리카노 2 종류 있습니다 \n']
p:  ['네 캐리어 에 담아 드리겠습니다 \n']
_
q:  ['카카오 페이 로 결제 가능한가요 \n']
a:  ['\t 네 

_
q:  ['얼마 에요 \n']
a:  ['\t 만 원 입니다 \n']
p:  ['베이글 과 동일하게 2000원 입니다 \n']
_
q:  ['아이스 아메리카노 랑 샌드위치 주세요 \n']
a:  ['\t 10시 에 세트 할인 가능하세요 \n']
p:  ['아메리카노 아메리카노 에 한 잔 로 드리겠습니다 \n']
